## Pré-processamento de texto e divisão de dados de treino e teste

### Imports

In [1]:
import pandas as pd
import numpy as np

from sklearn.naive_bayes import MultinomialNB
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.pipeline import make_pipeline
from sklearn import metrics
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression

#### Imports pré-processamento

In [2]:
import re
import nltk
import matplotlib.pyplot as plt
from nltk.stem import SnowballStemmer
nltk.download('stopwords')
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Carregando os dados

In [3]:
df = pd.read_csv('./data/imdb-reviews-pt-br.csv')

### Manipulação dos dados

In [4]:
del df['id']
del df['text_en']

In [5]:
df_transformed = df.copy()
df_transformed.sentiment = df_transformed['sentiment'].map({'pos': 1, 'neg': 0})

In [6]:
def corrigir_nomes(nome):
    nome = nome.replace(r'[,;&!?/.:]', ' ').replace('(\\d|\\W)+|\w*\d\w*', ' ').replace(';', ' ').replace(':', ' ').replace(',', ' ').replace('"', ' ').replace('ç', 'c').replace('é', 'e').replace('ç', 'c').replace('ã', 'a').replace('ê', 'e').replace('í', 'i').replace('á', 'a').replace('ó', 'o').replace('ú', 'u').replace('â', 'a').replace('ô', 'o').replace(' o ', ' ').replace(' a ', ' ').replace(' os ', ' ').replace(' as ', ' ').replace(' um ', ' ').replace(' uma ', ' ').replace(' uns ', ' ').replace(' umas ', ' ').replace(' ao ', ' ').replace(' aos ', ' ').replace(' à ', ' ').replace(' às ', ' ').replace(' da ', ' ').replace(' das ', ' ').replace(' do ', ' ').replace(' dos ', ' ').replace(' na ', ' ').replace(' nas ', ' ').replace(' no ', ' ').replace(' num ', ' ').replace(' numas ', ' ')
    return nome

df_transformed['text_pt'] = df_transformed['text_pt'].apply(corrigir_nomes)

In [7]:
df_transformed['text_pt'] = df_transformed['text_pt'].dropna()

In [8]:
df_transformed['text_pt'] = df_transformed['text_pt'].str.lower()

In [9]:
e_words = df_transformed['text_pt']
sbs = SnowballStemmer("portuguese")

In [10]:
for w in e_words:
    rootWord = sbs.stem(w)

In [11]:
df_transformed['text_pt_without_stemming'] = df_transformed['text_pt'].apply(lambda x: ' '.join([word for word in x.split() if word not in (rootWord)]))
df_transformed.head()

,text_pt,sentiment,text_pt_without_stemming
0,mais vez sr. costner arrumou filme por muito ...,0,mais vez sr. costner arrumou por muito mais te...
1,este e exemplo motivo pelo qual maioria filmes...,0,este exemplo motivo pelo qual maioria mesmos. ...
2,primeiro de tudo eu odeio esses raps imbecis ...,0,primeiro tudo eu odeio raps imbecis poderiam a...
3,nem mesmo beatles puderam escrever musicas que...,0,nem beatles puderam escrever musicas todos gos...
4,filmes de fotos de latao nao e palavra apropri...,0,fotos latao palavra apropriada verdade tanto o...


In [12]:
stop_words = set(stopwords.words("portuguese"))
print(len(stop_words))

204


In [13]:
df_transformed.colums = ['text_pt']

df_transformed['text_pt_without_stopwords'] = df_transformed['text_pt'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
df_transformed.head()

c:\users\felipe\anaconda3\envs\checklist3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


,text_pt,sentiment,text_pt_without_stemming,text_pt_without_stopwords
0,mais vez sr. costner arrumou filme por muito ...,0,mais vez sr. costner arrumou por muito mais te...,vez sr. costner arrumou filme tempo necessario...
1,este e exemplo motivo pelo qual maioria filmes...,0,este exemplo motivo pelo qual maioria mesmos. ...,exemplo motivo maioria filmes acao sao mesmos....
2,primeiro de tudo eu odeio esses raps imbecis ...,0,primeiro tudo eu odeio raps imbecis poderiam a...,primeiro tudo odeio raps imbecis nao poderiam ...
3,nem mesmo beatles puderam escrever musicas que...,0,nem beatles puderam escrever musicas todos gos...,beatles puderam escrever musicas todos gostass...
4,filmes de fotos de latao nao e palavra apropri...,0,fotos latao palavra apropriada verdade tanto o...,filmes fotos latao nao palavra apropriada verd...


### Divisão treino e teste

In [14]:
df_train, df_test = train_test_split(
    df_transformed,
    test_size = 0.3, 
    random_state = 42
)

#### Salvando dados de test e train

In [15]:
df_train.to_csv('./data_train_test/df_train_imbd.csv')
df_test.to_csv('./data_train_test/df_test_imbd.csv')

#### Lendo dados de test e train

In [16]:
df_train = pd.read_csv('./data_train_test/df_train_imbd.csv')
df_test = pd.read_csv('./data_train_test/df_test_imbd.csv')